<a href="https://colab.research.google.com/github/Mohamed-Maher5/sms-classification/blob/main/sms_classification_bert_base_cased_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers[torch]
! pip install transformers
! pip install evaluate

In [3]:
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import confusion_matrix
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [4]:
training_args = TrainingArguments(output_dir="test_trainer")

In [5]:
metric = evaluate.load("accuracy")

In [6]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [7]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [8]:
df = pd.read_csv("/content/sms_spam_collection.csv")

X_train, X_temp, y_train, y_temp = train_test_split(df['sms'], df['label'], test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

train_data = pd.concat([X_train, y_train], axis=1)
val_data = pd.concat([X_val, y_val], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

train_data = Dataset.from_pandas(train_data)
train_data = train_data.remove_columns("__index_level_0__")

val_data = Dataset.from_pandas(val_data)
val_data = val_data.remove_columns("__index_level_0__")

test_data = Dataset.from_pandas(test_data)
test_data = test_data.remove_columns("__index_level_0__")

In [9]:
train_data,val_data,test_data

(Dataset({
     features: ['sms', 'label'],
     num_rows: 3344
 }),
 Dataset({
     features: ['sms', 'label'],
     num_rows: 1115
 }),
 Dataset({
     features: ['sms', 'label'],
     num_rows: 1115
 }))

In [10]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["sms"], padding="max_length", truncation=True)

tokenized_train_datasets = train_data.map(tokenize_function, batched=True)
tokenized_val_datasets = val_data.map(tokenize_function, batched=True)
tokenized_test_datasets = test_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/3344 [00:00<?, ? examples/s]

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

In [11]:
tokenized_train_datasets,tokenized_val_datasets,tokenized_test_datasets

(Dataset({
     features: ['sms', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 3344
 }),
 Dataset({
     features: ['sms', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 1115
 }),
 Dataset({
     features: ['sms', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 1115
 }))

In [12]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_val_datasets,
    compute_metrics=compute_metrics,
)

In [14]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.041173,0.992825
2,0.100700,0.049027,0.991031
3,0.017700,0.055824,0.991928


TrainOutput(global_step=1254, training_loss=0.048037937953711696, metrics={'train_runtime': 1052.5981, 'train_samples_per_second': 9.531, 'train_steps_per_second': 1.191, 'total_flos': 2639601205198848.0, 'train_loss': 0.048037937953711696, 'epoch': 3.0})

In [15]:
trainer.evaluate(tokenized_test_datasets)

{'eval_loss': 0.05950107425451279,
 'eval_accuracy': 0.9928251121076234,
 'eval_runtime': 35.8258,
 'eval_samples_per_second': 31.123,
 'eval_steps_per_second': 3.908,
 'epoch': 3.0}

In [16]:
trainer.predict(tokenized_test_datasets)

PredictionOutput(predictions=array([[ 8.603652 , -2.215831 , -2.6623542, -2.5266395, -2.4691238],
       [ 8.620933 , -2.1386409, -2.7510548, -2.603649 , -2.5175674],
       [ 8.5855465, -2.3378322, -2.6364162, -2.4971042, -2.4544454],
       ...,
       [ 8.631931 , -1.9203548, -2.8616538, -2.690373 , -2.6446993],
       [ 8.297567 , -1.0777948, -3.239012 , -2.9611735, -3.092899 ],
       [ 8.61496  , -2.1606112, -2.7067115, -2.5728807, -2.524247 ]],
      dtype=float32), label_ids=array([0, 0, 0, ..., 0, 0, 0]), metrics={'test_loss': 0.05950107425451279, 'test_accuracy': 0.9928251121076234, 'test_runtime': 35.9235, 'test_samples_per_second': 31.038, 'test_steps_per_second': 3.897})

In [17]:
trainer.predict(tokenized_test_datasets)[1].shape

(1115,)

In [18]:
test_outputs = trainer.predict(tokenized_test_datasets)[1]

In [19]:
test_outputs[0]

0

In [20]:
cm=confusion_matrix(tokenized_test_datasets['label'],test_outputs)
cm

array([[960,   0],
       [  0, 155]])

In [21]:
trainer.save_model('sms_classification_model')

In [22]:
model_path = "/content/sms_classification_model"
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [25]:
new_text = "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005."
inputs = tokenizer(new_text, return_tensors="pt")

with torch.no_grad():
    predicted = model(**inputs)

class_probabilities = torch.softmax(predicted.logits, dim=-1)
predicted_class = torch.argmax(class_probabilities, dim=-1).item()

In [26]:
if(predicted_class == 0):
  print(f"Predicted Class: Ham")
else:
  print(f"Predicted Class: Spam")

Predicted Class: Spam
